In [1]:
from pathlib import Path

In [2]:
evaluation_path = str(Path.home() / "temp" / "iceberg" / "evaluation")
warehouse_path = str(Path.home() / "temp" / "iceberg" / "evaluation" / "spark-warehouse")
catalog_name = "local"

In [ ]:
from sedona.spark import *

config = (
    SedonaContext.builder()
    .config(
        "spark.jars.packages",
        "org.apache.sedona:sedona-spark-3.5_2.12:1.7.1,"
        "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.9.0,"
        "org.datasyslab:geotools-wrapper:1.7.1-28.5"
    )
    .config(
        "spark.jars.repositories",
        "https://artifacts.unidata.ucar.edu/repository/unidata-all",
    )
    .config(
        "spark.sql.extensions",
        "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
    )
    .config(
        f"spark.sql.catalog.{catalog_name}",
        "org.apache.iceberg.spark.SparkCatalog"
    )
    .config(
        f"spark.sql.catalog.{catalog_name}.type", "hadoop"
    )
    .config(
        f"spark.sql.catalog.{catalog_name}.warehouse",
        f"{warehouse_path}/{catalog_name}"
    )
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .config("spark.sql.session.timeZone", "UTC")
    .config("spark.driver.host", "localhost")
    .config("spark.driver.bindAddress", "localhost")
    .config("spark.driver.memory", "16g")
    .getOrCreate()
)
sedona = SedonaContext.create(config)

https://artifacts.unidata.ucar.edu/repository/unidata-all added as a remote repository with the name: repo-1
Ivy Default Cache set to: /Users/mdenno/.ivy2/cache
The jars for the packages stored in: /Users/mdenno/.ivy2/jars
org.apache.sedona#sedona-spark-3.5_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8f20ae39-f836-476f-8a73-64e8f2c20481;1.0
	confs: [default]
	found org.apache.sedona#sedona-spark-3.5_2.12;1.7.1 in central
	found org.apache.sedona#sedona-common;1.7.1 in central
	found org.apache.commons#commons-math3;3.6.1 in central


:: loading settings :: url = jar:file:/Users/mdenno/repos/teehr/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.locationtech.jts#jts-core;1.20.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found org.locationtech.spatial4j#spatial4j;0.8 in central
	found com.google.geometry#s2-geometry;2.0.0 in central
	found com.google.guava#guava;25.1-jre in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found org.checkerframework#checker-qual;2.0.0 in central
	found com.google.errorprone#error_prone_annotations;2.1.3 in central
	found com.google.j2objc#j2objc-annotations;1.1 in central
	found org.codehaus.mojo#animal-sniffer-annotations;1.14 in central
	found com.uber#h3;4.1.1 in central
	found net.sf.geographiclib#GeographicLib-Java;1.52 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.2 in central
	found org.checkerframework#checker-qual;3.10.0 in central
	found com.google.errorprone#error_prone_annotations;2.5.1 in central
	found org.apache.sedona#sedona-spark-common-3.5_2.12;1.7.1 in central
	found org.apache.sedona#shade-proto;1.7.1 in central
	foun

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 52212)
Traceback (most recent call last):
  File "/Users/mdenno/.pyenv/versions/3.10.15/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/mdenno/.pyenv/versions/3.10.15/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/mdenno/.pyenv/versions/3.10.15/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/mdenno/.pyenv/versions/3.10.15/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/mdenno/repos/teehr/.venv/lib/python3.10/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/Users/mdenno/repos/teehr/.venv/lib/python3.10/site-packages/pyspark/accumulators.py", line 26

In [4]:
sedona.sql(f"""
    SELECT * FROM local.db.primary_timeseries
    WHERE location_id == 'usgs-14182500'
    ORDER BY value_time ASC
    LIMIT 10
""").show()

+--------------+-------------------+------------------+---------+--------------------+---------+-------------+
|reference_time|         value_time|configuration_name|unit_name|       variable_name|    value|  location_id|
+--------------+-------------------+------------------+---------+--------------------+---------+-------------+
|          NULL|1989-01-01 00:00:00| usgs_observations|    m^3/s|streamflow_daily_...|24.823256|usgs-14182500|
|          NULL|1989-01-02 00:00:00| usgs_observations|    m^3/s|streamflow_daily_...|21.668287|usgs-14182500|
|          NULL|1989-01-03 00:00:00| usgs_observations|    m^3/s|streamflow_daily_...|31.285395|usgs-14182500|
|          NULL|1989-01-04 00:00:00| usgs_observations|    m^3/s|streamflow_daily_...|38.038963|usgs-14182500|
|          NULL|1989-01-05 00:00:00| usgs_observations|    m^3/s|streamflow_daily_...| 37.80299|usgs-14182500|
|          NULL|1989-01-06 00:00:00| usgs_observations|    m^3/s|streamflow_daily_...|29.954504|usgs-14182500|
|

In [5]:
attribute_names = [row.attribute_name for row in sedona.sql(f"""
    SELECT DISTINCT(attribute_name) FROM local.db.location_attributes
""").collect()]
attribute_names_sql = ", ".join([f"'{name}'" for name in attribute_names])

sedona.sql(f"""
    CREATE OR REPLACE TEMP VIEW locations_view AS (
        WITH location_attributes_pivot AS (
            SELECT *
            FROM (
                SELECT location_id, attribute_name, value
                FROM local.db.location_attributes
            ) src
            PIVOT (
                max(value) FOR attribute_name IN ({attribute_names_sql})
            )
        )
        SELECT l.id, l.name, ST_GeomFromWKB(l.geometry) AS geometry, la.*
        FROM local.db.locations l
        LEFT JOIN location_attributes_pivot la
        ON l.id = la.location_id
        WHERE l.id IS NOT NULL
        AND la.location_id IS NOT NULL
    )
""")

25/06/01 09:44:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[]

In [6]:
sedona.sql(f"""
    SELECT *
    FROM locations_view
    WHERE id = 'usgs-01013500'
""").show()

+-------------+--------------------+--------------------+-------------+----------------+-----------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+----------------+-------------------+----------------+----------+-------------+----------------+---------+----------+-----------+-----------+-----------+---------------+--------------+--------------------+-----------------+------------+---------------------+
|           id|                name|            geometry|  location_id|         aridity|    p_seasonality|                q5|              q95|     runoff_ratio|            q_mean|        frac_snow|   baseflow_index|    soil_porosity|       slope_fdc|dom_land_cover_frac|        pet_mean|slope_mean|drainage_area|          p_mean|elev_mean|frac_urban|zero_q_freq|high_q_freq|forest_frac|NID_dam_lengths|high_prec_freq|        ecoregion_L2|   dom_land_cover|stream_order|river_forecast_center|
+-------------

In [7]:
sedona.sql("""
CREATE OR REPLACE TEMP VIEW joined_timeseries_view AS (
    SELECT
        sf.reference_time
        , sf.value_time as value_time
        , pf.location_id as primary_location_id
        , sf.location_id as secondary_location_id
        , pf.value as primary_value
        , sf.value as secondary_value
        , sf.configuration_name
        , sf.unit_name
        , sf.variable_name
        , sf.member
    FROM local.db.secondary_timeseries sf
    JOIN local.db.location_crosswalks cf
        on cf.secondary_location_id = sf.location_id
    JOIN local.db.primary_timeseries pf
        on cf.primary_location_id = pf.location_id
        and sf.value_time = pf.value_time
        and sf.unit_name = pf.unit_name
        and sf.variable_name = pf.variable_name
)
""", )


DataFrame[]

In [8]:
sdf = sedona.sql("""
    SELECT river_forecast_center, avg(primary_value) as ave FROM joined_timeseries_view jtv
        JOIN locations_view lv
        ON jtv.primary_location_id = lv.id
    GROUP BY river_forecast_center;
""")
sdf.show()

+---------------------+------------------+
|river_forecast_center|               ave|
+---------------------+------------------+
|                CBRFC|1.5549045075001895|
|                NWRFC| 24.37332440867924|
|                NCRFC|10.509092417281442|
|                WGRFC|3.0123704995564933|
|                MARFC| 6.642655581080046|
|                NERFC|11.451879210220044|
|                LMRFC|14.989565046280838|
|                OHRFC|  9.97940754219624|
|                ABRFC|3.4584774255171933|
|                SERFC| 6.083878497829004|
|                CNRFC|12.503939524957193|
|                MBRFC| 7.807546943110796|
+---------------------+------------------+



In [ ]:
df = sedona.createDataFrame(
    [
        ('NULL', "1989-01-01 00:00:00", "usgs_observations", "m^3/s", "streamflow_daily_mean", 24.823256, "usgs-14182500"),
        ('NULL', "1989-01-02 00:00:00", "usgs_observations", "m^3/s", "streamflow_daily_mean", 21.668287, "usgs-14182500"),
    ],
    ["reference_time", "value_time", "configuration_name", "unit_name", "variable_name", "value", "location_id"]
)
sedona.sql("SELECT * FROM {df}", df=df).show()

+--------------+-------------------+------------------+---------+--------------------+---------+-------------+
|reference_time|         value_time|configuration_name|unit_name|       variable_name|    value|  location_id|
+--------------+-------------------+------------------+---------+--------------------+---------+-------------+
|          NULL|1989-01-01 00:00:00| usgs_observations|    m^3/s|streamflow_daily_...|24.823256|usgs-14182500|
|          NULL|1989-01-02 00:00:00| usgs_observations|    m^3/s|streamflow_daily_...|21.668287|usgs-14182500|
+--------------+-------------------+------------------+---------+--------------------+---------+-------------+



25/06/01 15:45:49 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 306789 ms exceeds timeout 120000 ms
25/06/01 15:45:49 WARN SparkContext: Killing executors is not supported by current scheduler.
25/06/01 15:45:52 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [ ]:
sedona.stop()

- Create joined timeseries needs some filters so that we can incrementally create the joined timeseries.  
- Joined timeseries should maybe only contain the joined ts.  That join process does not parallelize well.  Other joins do.